# Homework 3

There are three homework problems.  If a cell is blank or starts with a `# MODIFY` comment, please fill in the appropriate code.  Feel free to add cells as needed (`ESC-a` and `ESC-b`).

## Problem 1 - EDA

Visit www.kaggle.com and create an account.  Identify a dataset (https://www.kaggle.com/datasets) that interests you, and that also has:

* at least 2 numeric columns
* at least 1 categorical column

Add the import statement and "magic" command needed to use pandas and to plot in the notebook.

Read your data set into a pandas dataframe.

Create two lists named `numeric_cols` and `nonnumeric_cols` that contain the column labels for the numeric and non-numeric columns of the data set, respectively.  You should do this _programmatically_, not by hand.

Create a histogram with 20 bins for one of your numeric columns (your choice).

Generate a correlation table for your numeric columns.  How many columns are highly correlated (correlation above, say, 0.5)?

Create a scatter plot with one numeric column on the x-axis and another on the y-axis.  Pick a highly correlated pair of columns, if possible (if this is not possible, just select two columns of your choice).  Give the plot a title and set its size to be 8-by-5 inches.

Use the `groupby` method to create a dataframe of the frequency of each level of one of your categorical columns (your choice).  Then sort by size (greatest frequency to smallest) and keep only the top 10 most frequent categories.  If you have fewer than 10 levels then just keep all of them.

Plot the results of the groupby using a bar chart.

## Problem 2 - Optimization Modeling Exercise 2.2

In this problem you will solve textbook exercise 2.2 which was on Homework 1.  Your are given partial code and just need to fill in the missing pieces to get the model to work.  The solution to this problem can be found in the Excel file that accompanies this homework assignment.

__NOTE:__ These models are very similar to the simple LP we worked in class (see notebook in this directory), except here you are reading data in from a file instead of manually typing it.  If you are unsure about what is being asked for, first look at that example and see if you can find the analogous step!

### Step 1:  Read parameters from Excel file

Read in the the entire first sheet into a dataframe named `raw_data` from the model spreadsheet.  Do not use offsets or skip rows, just read in the entire sheet.

This next cell extracts the coefficients of the objective function (revenue) and the LHS of the resource constraints for the various ingredients.  You do not need to modify this cell, assuming you followed instructions in the previous cell.

In [ ]:
# RUN BUT DO NOT MODIFY
coef = pd.DataFrame(raw_data.iloc[[4, 7, 8, 9, 10, 11], 1:5])
coef.index = ['revenue', 'dough', 'sauce', 'cheese', 'meat', 'veggies']
coef.columns = ['plain', 'meat', 'veggie', 'supreme']
coef

The next cell reads in the RHS of the constraints into their own dataframe.  You do not need to modify this cell.

In [ ]:
# RUN BUT DO NOT MODIFY
rhs = pd.DataFrame(raw_data.iloc[7:12, 8])
rhs.index = coef.index[1:]
rhs.columns = ['rhs']
rhs

### Create a Pyomo model

In this section you will instantiate the decision variables, objective function and constraints for a Pyomo model that solves Exercise 2.2.

First, import the pyomo.environ module as `pe`.

Next, instantiate a ConcreteModel and store it in the variable `model`.

Create a list called `decision_index` containing indexes for the decision variables ranging from 0 to 3.

Create a pyomo variable named `x` with domain of nonnegative of real numbers.  Make sure you "attach" this variable to the `model` object.

The next cell defines the objective function `obj` using the data in the `coefficients` dataframe.  The expression argument, `expr=...`, shows a shorthand way to represent a sumproduct using a "list comprehension".  The `sense=-1` argument tells Pyomo that you want to maximize (`sense=1` is minimization, the default).

In [ ]:
# RUN BUT DO NOT MODIFY
model.obj = pe.Objective(expr=sum([coef.iloc[0,i]*model.x[i] for i in decision_index]),
                         sense=-1)

Next we need to define the five constraints using the data in the `coefficients` and `rhs` dataframes.  You've been given the first constraint for "dough" which shows how to use a list comprehension to calculate the LHS sumproduct.  Try to reuse this expression to create the other four constraints.

In [ ]:
model.cons_dough = pe.Constraint(expr=sum([coef.iloc[1, i]*model.x[i] for i in decision_index]) <= rhs.iloc[0, 0])
# MODIFY:  Write the other four constraints here

### Run the solver and examine the solution

Let's try to solve!  You do not need to modify this cell, unless you want to turn off the printed output (set `tee=False`).

In [ ]:
opt = pe.SolverFactory('glpk')
success = opt.solve(model, tee=True)

This next cell will help you determine whether the solver found a solution or had an error (like poorly defined constraints, or infeasibility).  Pay particular attention to the "Solver" part of the output.

In [ ]:
print(success)

The next cell will extract the objective function and optimal solution (assuming the solver found a solution).  You do not need to modify this cell.

In [ ]:
# RUN BUT DO NOT MODIFY
obj_val = model.obj.expr()
print('optimal objective value = {}'.format(obj_val))

x = []
for index in model.x_index.value_list:
    x.append(model.x[index].value)
print('optimal x = {}'.format(x))

Finally, print the slack in each constraint.  The expression to print the slack in the "dough" constraint is given.

In [ ]:
print('dough slack = {}'.format(model.cons_dough.slack()))
# MODIFY:  print the slack for the other four variables here.

## Problem 3 - Optimization Modeling Exercise 6.5

See if you can do this one yourself.  To receive full credit, you must read data into two dataframes (like `coef` and `rhs` in the previous problem) and use this to define the objective and constraints.